# Prediction on Google ML with Tensorflow

### Note

- Ensure the same version for Tensorflow is used locally and on Google cloud

https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list

## Setup

### Configuration

In [1]:
ROOT_PATH = '../services/prediction-tensorflow'

MODEL_PATH = '%s/models' % (ROOT_PATH)
EXPORTED_MODEL_PATH = '%s/tf_exported_model' % (MODEL_PATH)
EXPORTED_YSCALAR_PATH = '%s/yscaler.dat' % (MODEL_PATH)

DATA_PATH = '%s/data' % (ROOT_PATH)
DATA_INPUT_JSON_PATH = '%s/sample_input_prescaled.json' % (DATA_PATH)

BUCKET_NAME = 'tf_earning-prediction-1000'
BUCKET_FOLDER_NAME = 'earnings_v1'
BUCKET_URL = 'gs://%s' % (BUCKET_NAME)
BUCKET_FOLDER_URL = '%s/%s' % (BUCKET_URL, BUCKET_FOLDER_NAME)

MODEL_NAME = 'tf_earnings_prediction'
MODEL_VERSION = 'v1'
MODEL_RUNTIME_VERSION = '1.2'

PROJECT_ID = "gde-core-dev"
REGION = 'us-central1'
CREDENTIALS_FILE = "./credentials.json"

### Libraries

- google-api-python-client
- oauth2client
- pickle

In [2]:
from oauth2client.client import GoogleCredentials
import googleapiclient.discovery

import pickle

### Upload the model to Google Cloud bucket

In [3]:
# Make the bucket
!gsutil mb -l { REGION } { BUCKET_URL }
!gsutil cp -r { EXPORTED_MODEL_PATH } { BUCKET_FOLDER_URL }


Creating gs://tf_earning-prediction-1000/...
Copying file://../services/prediction-tensorflow/models/tf_exported_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://../services/prediction-tensorflow/models/tf_exported_model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://../services/prediction-tensorflow/models/tf_exported_model/variables/variables.index [Content-Type=application/octet-stream]...
- [3 files][195.4 KiB/195.4 KiB]                                                
Operation completed over 3 objects/195.4 KiB.                                    


### Create the model with Google ML Engine

In [4]:
!gcloud ml-engine models create { MODEL_NAME } --regions { REGION }

Created ml engine model [projects/gde-core-dev/models/tf_earnings_prediction].


In [5]:
!gcloud ml-engine versions create { MODEL_VERSION } \
    --model={ MODEL_NAME } \
    --origin={ BUCKET_FOLDER_URL }/ \
    --runtime-version={ MODEL_RUNTIME_VERSION }


Creating version (this might take a few minutes)......done.                    


## Prediction

### Input

In [6]:
input = [[0.5, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0]]

In [7]:
yscaler = pickle.load(open(EXPORTED_YSCALAR_PATH, 'rb'))

### Predict with local saved_model_cli

In [8]:
output = !saved_model_cli run \
    --dir { EXPORTED_MODEL_PATH } \
    --tag_set serve --signature_def serving_default \
    --input_exprs 'input={ input }'

print(output)

['2018-10-25 11:56:13.623732: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA', 'Result for output key earnings:', '[[0.78507453]]']


In [9]:
test_scaled_set = yscaler.inverse_transform(eval(output[len(output) - 1]))

print("Our neural network predicted earnings of ${}".format(test_scaled_set[0][0]))

Our neural network predicted earnings of $243721.58573765002


### Predict from Google ML Engine using gcloud

In [10]:
output = !gcloud ml-engine predict \
    --model={ MODEL_NAME } \
    --json-instances={ DATA_INPUT_JSON_PATH }

print(output)

['EARNINGS', '[0.7850745320320129]']


In [11]:
test_scaled_set = yscaler.inverse_transform([eval(output[1])])

print("Our neural network predicted earnings of ${}".format(test_scaled_set[0]))

Our neural network predicted earnings of $[243721.58628732]


### Predict from Google ML Engine using SDK

In [12]:
# These are the values we want a prediction for
inputs_for_prediction = [{
    "input": [0.5, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0]
}]

# Connect to the Google Cloud-ML Service
credentials = GoogleCredentials.from_stream(CREDENTIALS_FILE)
service = googleapiclient.discovery.build('ml', 'v1', credentials=credentials)

# Connect to our Prediction Model
name = 'projects/{}/models/{}'.format(PROJECT_ID, MODEL_NAME)
response = service.projects().predict(
    name=name,
    body={'instances': inputs_for_prediction}
).execute()

# Report any errors
if 'error' in response:
    raise RuntimeError(response['error'])

# Grab the results from the response object
results = response['predictions']

# Print the results!
print(results)

[{'earnings': [0.7850745320320129]}]


## Tear Down

### Delete the model from Google ML Engine

In [13]:
!gcloud ml-engine versions delete { MODEL_VERSION } \
    --model={ MODEL_NAME } \
    --quiet
!gcloud ml-engine models delete { MODEL_NAME } --quiet

Deleting version [v1]......done.                                               
Deleting model [tf_earnings_prediction]...done.                                


In [14]:
!gsutil -m rm -r { BUCKET_URL } 

Removing gs://tf_earning-prediction-1000/earnings_v1/saved_model.pb#1540493294954930...
Removing gs://tf_earning-prediction-1000/earnings_v1/variables/variables.data-00000-of-00001#1540493295365738...
Removing gs://tf_earning-prediction-1000/earnings_v1/variables/variables.index#1540493295593113...
/ [3/3 objects] 100% Done                                                       
Operation completed over 3 objects.                                              
Removing gs://tf_earning-prediction-1000/...


## Troubleshooting

In [15]:
# Check the model signature with saved_model_cli

!saved_model_cli show --dir { EXPORTED_MODEL_PATH }  --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 9)
        name: input/Placeholder:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['earnings'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: output/add:0
  Method name is: tensorflow/serving/predict


In [16]:
# Test prediction with saved_model_cli

!saved_model_cli run \
    --dir { EXPORTED_MODEL_PATH } \
    --tag_set serve \
    --signature_def serving_default \
    --input_exprs 'input={ input }'

2018-10-25 11:57:35.134554: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Result for output key earnings:
[[0.78507453]]
